# Apply RNAseg on spatial data 

In [15]:
import spatialdata as sd
from pathlib import Path
from rna_seg.dataset_zarr.patches import create_patch_rnaseg
from rna_seg.dataset_zarr.RNAsegDataset import RNAsegDataset
from rna_seg.models import RNASeg
import albumentations as A
import cv2

# 1) Create datasetfrom a spatialdata

In [16]:
### load sdata and set path parameters 
merfish_zarr_path = "/media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr"
sdata = sd.read_zarr(merfish_zarr_path)

image_key = "staining_z3"
points_key = "transcripts"

patch_width = 1200
patch_overlap = 150



min_transcripts_per_patch = 0
folder_patch_rna_seg = Path(merfish_zarr_path) / ".rna_seg"

### create patch in the sdata and precompute transcipt.csv for each patch with sopa
create_patch_rnaseg(sdata,
                    image_key=image_key,
                    points_key=points_key,
                    patch_width=patch_width,
                    patch_overlap=patch_overlap,
                    min_transcripts_per_patch=min_transcripts_per_patch,
                    overwrite = True)
print(sdata)

/home/tom/anaconda3/envs/RNAseg/lib/python3.10/site-packages/spatialdata/_core/_elements.py:96: UserWarning: Key `sopa_patches_rna_seg_1200_150` already exists. Overwriting it in-memory.
  self._check_key(key, self.keys(), self._shared_keys)
[INFO] (sopa.patches.patches) 81 patches were saved in sdata['sopa_patches_rna_seg_1200_150']


[########################################] | 100% Completed | 21.66 ss
SpatialData object, with associated Zarr store: /media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/test_mouse_ileum.zarr
├── Images
│     └── 'staining_z3': DataTree[cyx] (5, 9000, 9000), (5, 4500, 4500), (5, 2250, 2250), (5, 1125, 1125), (5, 562, 562)
├── Points
│     └── 'transcripts': DataFrame with shape: (<Delayed>, 9) (2D points)
└── Shapes
      ├── 'Cellbound1': GeoDataFrame shape: (3258, 1) (2D shapes)
      ├── 'Cellbound1_consistent_with_nuclei': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'Cellbound1_consistent_without_nuclei': GeoDataFrame shape: (2239, 1) (2D shapes)
      ├── 'DAPI': GeoDataFrame shape: (2377, 1) (2D shapes)
      ├── 'DAPI_consistent_in_cell': GeoDataFrame shape: (1007, 1) (2D shapes)
      ├── 'DAPI_consistent_not_in_cell': GeoDataFrame shape: (1370, 1) (2D shapes)
      ├── 'sopa_patches_rna_seg_1200_50': GeoDataFrame shape: (64, 3) (2D shapes)
      └── 'sopa

In [17]:
transform_resize  = A.Compose([
 A.Resize(width=512, height=512, interpolation=cv2.INTER_NEAREST),
])


dataset = RNAsegDataset(
sdata=sdata,
channels_dapi= ["DAPI"],
channels_cellbound=["Cellbound1"],
patch_width = 1200,
patch_overlap = 150,
gene_column="gene",
transform_resize = transform_resize,
 )

default shape_patch_key set to sopa_patches_rna_seg_1200_150
100%|██████████| 81/81 [00:02<00:00, 35.20it/s]
Number of valid patches: 64


# 2) run RNAseg

In [14]:

pretrained_model = "/media/tom/Transcend/open_merfish/test_spatial_data/from_cluster/small_merfish_for_test.zarr/model/best.pt"
device = "cpu"
rnaseg = RNASeg(device,
                net='unet',
                flow_threshold = 0.9,
                cellbound_flow_threshold = 0.4,
                pretrained_model = None,
                )

initiaisation of CPnet
Initiaisation of ChannelInvariantNet


# 3) Add new segmentation to the input spatial data

In [1]:
from tqdm import tqdm

for i in tqdm(list(range(len(dataset)))):
    input_dict = dataset[i]
    rnaseg.run(input_dict=input_dict)



NameError: name 'dataset' is not defined

In [30]:
input_dict

{'img_cellbound': tensor([[[0.2747, 0.3020, 0.3220,  ..., 0.3325, 0.3068, 0.3180],
          [0.3097, 0.3104, 0.3687,  ..., 0.3021, 0.3033, 0.3033],
          [0.3693, 0.3628, 0.3999,  ..., 0.3130, 0.3118, 0.3187],
          ...,
          [0.5089, 0.5189, 0.5173,  ..., 0.1041, 0.0901, 0.0854],
          [0.5269, 0.5066, 0.4928,  ..., 0.0898, 0.0822, 0.0870],
          [0.5466, 0.5504, 0.5039,  ..., 0.0865, 0.0892, 0.0929]]]),
 'dapi': tensor([[[ 0.3373,  0.3294,  0.3038,  ...,  0.1389,  0.1610,  0.1832],
          [ 0.3062,  0.2798,  0.2649,  ...,  0.1546,  0.1919,  0.2171],
          [ 0.2690,  0.2605,  0.2524,  ...,  0.1821,  0.2148,  0.2450],
          ...,
          [ 0.1838,  0.1742,  0.1682,  ..., -0.0051, -0.0066, -0.0033],
          [ 0.2102,  0.1960,  0.1944,  ..., -0.0060, -0.0096, -0.0100],
          [ 0.2275,  0.2186,  0.2045,  ..., -0.0099, -0.0047, -0.0136]]]),
 'rna_img': tensor([[[0.0000e+00, 0.0000e+00, 1.1416e-02,  ..., 0.0000e+00,
           9.5534e-02, 1.1444e-02],

In [20]:
input_dict.get('dapi', None)

tensor([[[ 0.3373,  0.3294,  0.3038,  ...,  0.1389,  0.1610,  0.1832],
         [ 0.3062,  0.2798,  0.2649,  ...,  0.1546,  0.1919,  0.2171],
         [ 0.2690,  0.2605,  0.2524,  ...,  0.1821,  0.2148,  0.2450],
         ...,
         [ 0.1838,  0.1742,  0.1682,  ..., -0.0051, -0.0066, -0.0033],
         [ 0.2102,  0.1960,  0.1944,  ..., -0.0060, -0.0096, -0.0100],
         [ 0.2275,  0.2186,  0.2045,  ..., -0.0099, -0.0047, -0.0136]]])